# 01_2. Formula 1 Qualifying Data Collection

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
start = time.time()

## Formula 1 Races from 1983 until 2022

In [ ]:
qualifying_results = pd.DataFrame()
for year in list(range(1983,2023)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_results = pd.concat([qualifying_results, year_df])
    
print(qualifying_results.shape)

In [ ]:
len(qualifying_results)

In [ ]:
qualifying_results.head()

In [ ]:
qualifying_results.tail()

In [ ]:
qualifying_results.rename(columns = {'Pos': 'grid_position', 'Driver': 'driver_name', 'Car': 'car',
                                     'Time': 'qualifying_time'}, inplace = True)

In [ ]:
qualifying_results.drop('No', axis = 1, inplace = True)

In [ ]:
qualifying_results.to_csv('./data/qualifying.csv', index = False)

In [ ]:
end = time.time()

In [ ]:
import datetime
str(datetime.timedelta(seconds=(end - start)))

In [ ]:
print(str(end - start)+" seconds")